In [82]:
from typing import List, Tuple

import matplotlib.pyplot as plt

from elections.HeadToHeadElection import HeadToHeadElection
from elections.DefaultConfigOptions import unit_election_config
from Experiment import Experiment
from elections.ElectionConstructor import ElectionConstructor, construct_irv, construct_h2h
from elections.Candidate import Candidate
from elections.Ballot import Ballot

n_races=1000

h2h_config = ExperimentalConfig("H2H",
                            20000,
                            1.5, .7, 21, 512, 3, 50000, 2048, 400, 1000, "exp/h2h-1")
h2h_config.save()
h2h_exp = Experiment(h2h_config)

In [83]:
irv_config = ExperimentConfig("IRV",
                            20000,
                            1.5, .7, 21, 512, 3, 50000, 2048, 400, 1000, "exp/irv-1")
irv_config.save()
irv_exp = Experiment(irv_config)

In [ ]:
irv_wc_s = irv_exp.run_strategic_races_core(n_races)
HeadToHeadElection.count_of_ties = 0
h2h_wc_s = h2h_exp.run_strategic_races_core(n_races)
print(f"number of HeadToHead ties in strategic races: {HeadToHeadElection.count_of_ties}")

populating training memory with 250000 samples
...................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................
m.count 250000
training network for 100000 epochs
epoch     0 loss = 0.033196


2021-07-31 10:46:35.092589: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mdl.sav.0/assets
epoch  1000 loss = 0.0194754
INFO:tensorflow:Assets written to: mdl.sav.1000/assets
epoch  2000 loss = 0.0187895
INFO:tensorflow:Assets written to: mdl.sav.2000/assets
epoch  3000 loss = 0.0181654
INFO:tensorflow:Assets written to: mdl.sav.3000/assets
loss is nan, reverting to mdl.sav.3000


epoch  4000 loss = 0.0176996
INFO:tensorflow:Assets written to: mdl.sav.4000/assets


INFO:tensorflow:Assets written to: mdl.sav.4000/assets


loss is nan, reverting to mdl.sav.4000


loss is nan, reverting to mdl.sav.4000


epoch  5000 loss = 0.0177241
INFO:tensorflow:Assets written to: mdl.sav.5000/assets


INFO:tensorflow:Assets written to: mdl.sav.5000/assets


In [ ]:
import numpy as np
def plot_results(results: List[List[float]], title: str, labels: List[str]):
    n_rows = 1
    n_cols = 1
    fig, axis = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(20, 10))
    fig.suptitle(title, color="black", fontsize=22)
    fig.set_facecolor("white")

    count = 0
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    axis.tick_params(axis='x', colors="black")
    axis.tick_params(axis='y', colors="black")
    axis.set_xlim([-1, 1])

    bins = np.arange(-1, 1, 2/21)
    axis.hist(results, bins=bins, label=labels, edgecolor='white', stacked=True)
    axis.legend()
    axis.set_xlabel("Sigma From Origin", fontsize=20)
    axis.set_ylabel("Frequency of Winner at Ideology", fontsize=20)

    plt.savefig("foo.png")

In [ ]:
def results_for_candidate(results: List[Tuple[Candidate, List[Candidate]]], candidate_name: str, wins_only: bool):
    ideologies = []
    for w, cc in results:
        if wins_only and w.name == candidate_name:
            ideologies.append(w.ideology.vec[0])
        elif not wins_only:
            for c in cc:
                if c.name == candidate_name:
                    ideologies.append( c.ideology.vec[0] )

    print(f"found {len(ideologies)} results")
    return ideologies

In [ ]:
names = ["c-0", "c-1", "c-2", "c-3", "c-4"]
def make_plots():
    root="e/v1"
    results = [results_for_candidate(irv_wc_s, n, True) for n in names]
    plot_results(results, f"Frequency of Winning Ideology by Each Candidate With IRV", names)
    plt.savefig(f"{root}/wins_by_candidate_irv.png")

    results = [results_for_candidate(h2h_wc_s, n, True) for n in names]
    plot_results(results, f"Frequency of Winning Ideology by Each Candidate With Minimax", names)
    plt.savefig(f"{root}/wins_by_candidate_h2h.png")

    results = [results_for_candidate(irv_wc_s, n, False) for n in names]
    plot_results(results, f"Frequency of Chosen Ideology by Each Candidate With IRV", names)
    plt.savefig(f"{root}/chosen_by_candidate_irv.png")

    results = [results_for_candidate(h2h_wc_s, n, False) for n in names]
    plot_results(results, f"Frequency of Chosen Ideology by Each Candidate With Minimax", names)
    plt.savefig(f"{root}/chosen_by_candidate_h2h.png")

make_plots()

In [ ]:
def CumulativeNormalDistribution(xRaw: float) -> float:
    if xRaw < 0:
        neg = 1
    else:
        neg = 0

    k = 1.0 / (1.0 + 0.2316419 * abs(xRaw))
    y = ((((1.330274429 * k - 1.821255978) * k + 1.781477937) * k - 0.356563782) * k + 0.319381530) * k
    y = 1.0 - 0.398942280401 * np.exp(-0.5 * (xRaw * xRaw)) * y
    return (1.0 - neg) * y + neg * (1.0 - y)

In [ ]:
def representation(sigma: float) -> float:
    pct = CumulativeNormalDistribution(sigma)
    return 100 * (1 - 2 * abs(.5 - pct))

In [ ]:
def run_random_election(exp: Experiment) -> Tuple[Candidate, List[Candidate]]:
    candidates = exp.config.gen_candidates_2(5)
    # candidates = exp.config.gen_random_candidates(5)
    voters = exp.config.population.generate_unit_voters(exp.config.sampling_voters)
    ballots = [Ballot(v, candidates, unit_election_config) for v in voters]
    process = exp.config.election_constructor
    result = process.run(ballots, set(candidates))
    winner = result.winner()
    return winner, candidates, result.t

In [ ]:
abs(-.1)

In [ ]:
irv_wc_r = [run_random_election(irv_exp) for i in range(n_races)]
h2h_wc_r = [run_random_election(h2h_exp) for i in range(n_races)]
print(f"Number of HeadToHead ties in random races: {HeadToHeadElection.count_of_ties}")

In [ ]:
def mean(x: List[float]) -> float:
    return np.mean(np.array(x))

def describe_results(results: List[Tuple[Candidate, List[Candidate]]], label):
    winners = list(map(lambda x: x[0].ideology.vec[0], results))
    sigmas = [abs(x) for x in winners]
    percentages = [CumulativeNormalDistribution(x) for x in winners]
    rr = [representation(x) for x in winners]

    mean_sigma = mean(sigmas)
    mean_representation = mean(rr)

    print(f"%14s sigma from origin: %.3f representation %5.2f" % (label, mean_sigma, mean_representation))

In [ ]:
describe_results(irv_wc_r, "IRV/Random")
describe_results(h2h_wc_r, "H2H/Random")

describe_results(irv_wc_s, "IRV/Strategic")
describe_results(h2h_wc_s, "H2H/Strategic")
